In [1]:
import torch
import os
from PIL import Image
from pathlib import Path
import pandas as pd
import torchvision
from sklearn.model_selection import train_test_split

In [2]:
labels=pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
train,valid=train_test_split(labels,train_size=0.8,shuffle=True,stratify=labels['breed'],random_state=42)

In [3]:
train,train_labels=train['id'].reset_index(drop=True),train['breed'].reset_index(drop=True)
val,val_labels=valid['id'].reset_index(drop=True),valid['breed'].reset_index(drop=True)

In [44]:
'''Encoding labels'''

breeds=dict()
breed_count=1

for breed in labels['breed'].value_counts().index:

    breeds[breed]=breed_count-1
    breed_count+=1

val_labels_torch=torch.zeros(len(val_labels),1)
train_labels_torch=torch.zeros(len(train_labels),1)

for index in val_labels.index:
    val_labels_torch[index]=breeds[val_labels.iloc[index]]

for index in train_labels.index:
    train_labels_torch[index]=breeds[train_labels.iloc[index]]

val_labels_torch = val_labels_torch.long()
train_labels_torch = train_labels_torch.long()


val_labels_torch[91]
# val_labels=torch.tensor(val_labels.values)
# train_labels=torch.tensor(train_labels.values)

FileNotFoundError: [Errno 2] No such file or directory: 'val_pred.csv'

In [38]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(10),
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    torchvision.transforms.ToTensor()
])


def create_dataset(series, directory_path, transform):
    tensors = []
    for i in range(len(series)):
        if(series[i].endswith('.jpg')):
            img_file=series[i]
        else:
            img_file = series[i] + '.jpg'
        img_path = os.path.join(directory_path, img_file)
        img = Image.open(img_path).convert('RGB')  # Ensure RGB
        img_t = transform(img)
        tensors.append(img_t)
    return torch.stack(tensors, dim=0)


In [6]:
directory_path=Path('/kaggle/input/dog-breed-identification/train')
train_dataset=create_dataset(train,directory_path,transform)

In [7]:


# For validation: no augmentation
val_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor()
])
val_dataset = create_dataset(val, directory_path, transform=val_transform)


In [8]:
from torch.utils.data import TensorDataset

train_dataset_torch=TensorDataset(train_dataset,train_labels_torch)
val_dataset_torch=TensorDataset(val_dataset,val_labels_torch)

In [9]:
train_dataset.shape,train_labels_torch.shape

(torch.Size([8177, 3, 224, 224]), torch.Size([8177, 1]))

In [10]:
val_dataset_torch[0]

(tensor([[[0.2980, 0.2667, 0.1569,  ..., 0.2510, 0.0471, 0.0353],
          [0.2275, 0.1490, 0.2157,  ..., 0.1020, 0.0196, 0.0078],
          [0.2000, 0.1098, 0.2745,  ..., 0.0510, 0.0275, 0.0157],
          ...,
          [0.1176, 0.2118, 0.1843,  ..., 0.1176, 0.1373, 0.1451],
          [0.1490, 0.1843, 0.1804,  ..., 0.1765, 0.1569, 0.1451],
          [0.2431, 0.2784, 0.2118,  ..., 0.1529, 0.1216, 0.1059]],
 
         [[0.3765, 0.3373, 0.2275,  ..., 0.2863, 0.0549, 0.0314],
          [0.3137, 0.2314, 0.2980,  ..., 0.1373, 0.0275, 0.0039],
          [0.2941, 0.2039, 0.3647,  ..., 0.0824, 0.0392, 0.0157],
          ...,
          [0.1176, 0.2118, 0.1882,  ..., 0.1333, 0.1529, 0.1608],
          [0.1490, 0.1843, 0.1843,  ..., 0.1843, 0.1647, 0.1529],
          [0.2431, 0.2784, 0.2157,  ..., 0.1569, 0.1255, 0.1059]],
 
         [[0.3765, 0.3608, 0.2588,  ..., 0.2235, 0.0431, 0.0510],
          [0.2667, 0.2000, 0.2706,  ..., 0.0745, 0.0157, 0.0235],
          [0.1961, 0.1176, 0.2902,  ...,

In [11]:
#from torchvision.models import resnet101, ResNet101_Weights
import torch.nn as nn
import torch.optim as optim
#from torchvision.models import resnext50_32x4d, ResNeXt50_32X4D_Weights
from torchvision.models import resnext101_32x8d, ResNeXt101_32X8D_Weights
#resnet = resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)
resnet = resnext101_32x8d(weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1)
#resnet = resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)
for param in resnet.parameters():
    param.requires_grad = False


resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 256),  # From ResNet's output to 256 units
    nn.ReLU(),                              # Activation function
    nn.BatchNorm1d(256),                    # Normalization for stability
    nn.Linear(256, 256),                    # Additional dense layer
    nn.ReLU(),                              # Activation again
    nn.Dropout(0.3),                        # Dropout to reduce overfitting
    nn.BatchNorm1d(256),                    # Another batch norm layer
    nn.Linear(256, 120)                     # Final layer: 120 output classes
)
model_torch = resnet

optimizier=optim.Adam(model_torch.parameters(),lr=1e-4)
loss_fn=nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth
100%|██████████| 340M/340M [00:01<00:00, 195MB/s] 


In [12]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset_torch, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset_torch, batch_size=64, shuffle=False)

n_epochs=50
model_torch=model_torch.to('cuda')
for i in range(1,n_epochs+1):
    model_torch.train()
    training_loss=0.0
    for img,label in train_loader:
        img=img.to('cuda')
        label=label.to('cuda')
        label=label.squeeze(1)
        outputs=model_torch(img)
        optimizier.zero_grad()
        losses=loss_fn(outputs,label)
        training_loss+=losses.item()
        losses.backward()
        optimizier.step()

    model_torch.eval()
    val_loss=0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for img,label in val_loader:
            img=img.to('cuda')
            label=label.to('cuda')
            label=label.squeeze(1)
            outputs=model_torch(img)
            loss=loss_fn(outputs,label)
            val_loss+=loss.item()

            _, predicted = torch.max(outputs, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print(f"Epoch [{i}/{n_epochs}], Training Loss: {training_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")


Epoch [1/50], Training Loss: 3.6332, Validation Loss: 2.4814, Accuracy: 76.82%
Epoch [2/50], Training Loss: 2.2778, Validation Loss: 1.7070, Accuracy: 84.79%
Epoch [3/50], Training Loss: 1.6498, Validation Loss: 1.2711, Accuracy: 86.94%
Epoch [4/50], Training Loss: 1.2721, Validation Loss: 1.0257, Accuracy: 88.22%
Epoch [5/50], Training Loss: 1.0256, Validation Loss: 0.8421, Accuracy: 88.90%
Epoch [6/50], Training Loss: 0.8497, Validation Loss: 0.7478, Accuracy: 89.24%
Epoch [7/50], Training Loss: 0.7191, Validation Loss: 0.6599, Accuracy: 89.29%
Epoch [8/50], Training Loss: 0.6204, Validation Loss: 0.6201, Accuracy: 89.78%
Epoch [9/50], Training Loss: 0.5408, Validation Loss: 0.5727, Accuracy: 89.24%
Epoch [10/50], Training Loss: 0.4768, Validation Loss: 0.5340, Accuracy: 89.44%
Epoch [11/50], Training Loss: 0.4127, Validation Loss: 0.5047, Accuracy: 90.02%
Epoch [12/50], Training Loss: 0.3671, Validation Loss: 0.4832, Accuracy: 89.73%
Epoch [13/50], Training Loss: 0.3264, Validation 

In [13]:
torch.save(resnet.state_dict(), 'resnet_custom_head.pth')


In [14]:
os.listdir('/kaggle/input/dog-breed-identification/test')[1:10]

['f7a32505c12649183c5991ecfa7d68b3.jpg',
 'bc6d50ffb4644feb34530aa58943e85b.jpg',
 '7cbc041e79135a572aad87904b5c9c57.jpg',
 'f3b603f10d6a344f0395fb46b242ff8e.jpg',
 'db55098d859d27a3c05b0e3fc41d43e7.jpg',
 '665bab06a3ea861c0812c778486988e3.jpg',
 '7b57911ea7559cebd6db308125926642.jpg',
 'df0d6ba158287cb2b3ed6459a22d42ba.jpg',
 'b80d89ad4a17ac2537ff46d251efcd7c.jpg']

In [15]:

test=os.listdir('/kaggle/input/dog-breed-identification/test')
test_dataset=create_dataset(test,'/kaggle/input/dog-breed-identification/test',val_transform)
test_dataset=test_dataset.to(device='cuda')

In [26]:
import pandas as pd
output=pd.DataFrame(columns=['id',*breeds.keys()])

In [27]:
output

,id,scottish_deerhound,maltese_dog,afghan_hound,entlebucher,bernese_mountain_dog,shih-tzu,pomeranian,great_pyrenees,basenji,...,otterhound,giant_schnauzer,walker_hound,tibetan_mastiff,german_shepherd,golden_retriever,komondor,brabancon_griffon,eskimo_dog,briard


In [28]:
breeds

{'scottish_deerhound': 0,
 'maltese_dog': 1,
 'afghan_hound': 2,
 'entlebucher': 3,
 'bernese_mountain_dog': 4,
 'shih-tzu': 5,
 'pomeranian': 6,
 'great_pyrenees': 7,
 'basenji': 8,
 'samoyed': 9,
 'tibetan_terrier': 10,
 'airedale': 11,
 'cairn': 12,
 'leonberg': 13,
 'japanese_spaniel': 14,
 'beagle': 15,
 'blenheim_spaniel': 16,
 'miniature_pinscher': 17,
 'australian_terrier': 18,
 'irish_wolfhound': 19,
 'saluki': 20,
 'lakeland_terrier': 21,
 'papillon': 22,
 'norwegian_elkhound': 23,
 'siberian_husky': 24,
 'whippet': 25,
 'pug': 26,
 'chow': 27,
 'italian_greyhound': 28,
 'pembroke': 29,
 'newfoundland': 30,
 'ibizan_hound': 31,
 'border_terrier': 32,
 'silky_terrier': 33,
 'lhasa': 34,
 'dandie_dinmont': 35,
 'bedlington_terrier': 36,
 'sealyham_terrier': 37,
 'rhodesian_ridgeback': 38,
 'irish_setter': 39,
 'boston_bull': 40,
 'old_english_sheepdog': 41,
 'collie': 42,
 'english_foxhound': 43,
 'african_hunting_dog': 44,
 'bouvier_des_flandres': 45,
 'schipperke': 46,
 'kelp

In [29]:
import torch.nn.functional as F
i=0
for file,sample in zip(os.listdir('/kaggle/input/dog-breed-identification/test'),test_dataset):
    sample=sample.cuda()
    output_values=model_torch(sample.unsqueeze(dim=0))
    output_values=output_values.cpu()
    probabilities = F.softmax(output_values, dim=1)
    output.loc[i]=[file,*probabilities[0].detach().numpy()]
    i+=1

In [32]:
output.head()

,id,scottish_deerhound,maltese_dog,afghan_hound,entlebucher,bernese_mountain_dog,shih-tzu,pomeranian,great_pyrenees,basenji,...,otterhound,giant_schnauzer,walker_hound,tibetan_mastiff,german_shepherd,golden_retriever,komondor,brabancon_griffon,eskimo_dog,briard
0,09e51e17e2b756ff2ace8a87bd1443fa.jpg,6.025255e-05,0.000003,1.002169e-05,2.358841e-06,2.507272e-07,2.767077e-06,1.822788e-06,1.229310e-06,3.645274e-05,...,2.530872e-06,1.236663e-05,2.440093e-06,5.950207e-06,5.025217e-07,8.902872e-07,8.406071e-05,1.952949e-05,4.356022e-06,5.757016e-06
1,f7a32505c12649183c5991ecfa7d68b3.jpg,3.465677e-07,0.000003,2.536432e-06,5.143971e-07,5.161984e-08,6.022860e-07,5.494094e-07,1.764476e-06,2.138807e-06,...,1.974815e-07,8.269388e-07,4.782271e-06,7.331486e-07,6.025829e-07,8.528064e-07,4.971440e-07,2.226533e-06,1.177406e-06,8.392175e-07
2,bc6d50ffb4644feb34530aa58943e85b.jpg,1.023210e-05,0.000002,4.325002e-04,1.114045e-04,5.905761e-06,9.600379e-06,2.896341e-05,1.143488e-05,7.973370e-07,...,5.772430e-05,3.047230e-06,8.423241e-06,1.348264e-03,5.445114e-05,1.579936e-05,4.625443e-06,2.412243e-06,3.471185e-06,1.781174e-05
3,7cbc041e79135a572aad87904b5c9c57.jpg,1.384845e-05,0.000005,1.928310e-05,3.694186e-06,1.054745e-06,3.728694e-07,2.922924e-06,1.383700e-06,2.328253e-06,...,3.183350e-07,9.319788e-08,6.664375e-07,1.107835e-06,1.212698e-06,1.154219e-07,1.795224e-07,9.902078e-07,5.023959e-06,5.265103e-08
4,f3b603f10d6a344f0395fb46b242ff8e.jpg,1.990270e-08,0.000002,1.943796e-07,1.923286e-07,3.000916e-07,7.909387e-06,1.448787e-07,6.618707e-08,1.012216e-06,...,3.217189e-07,3.858731e-07,1.037855e-06,1.233188e-07,9.561035e-08,2.026149e-07,1.108814e-06,5.761114e-08,5.918504e-08,7.898694e-07


In [34]:
print(output.columns)

Index(['id', 'scottish_deerhound', 'maltese_dog', 'afghan_hound',
       'entlebucher', 'bernese_mountain_dog', 'shih-tzu', 'pomeranian',
       'great_pyrenees', 'basenji',
       ...
       'otterhound', 'giant_schnauzer', 'walker_hound', 'tibetan_mastiff',
       'german_shepherd', 'golden_retriever', 'komondor', 'brabancon_griffon',
       'eskimo_dog', 'briard'],
      dtype='object', length=121)


In [35]:
# Remove .jpg suffix before saving
output['id'] = output['id'].str.replace('.jpg', '', regex=False)

output.to_csv('/kaggle/working/test_submission.csv', index=False)

In [48]:
!pip list

Package                            Version             Editable project location
---------------------------------- ------------------- -------------------------
absl-py                            1.4.0
accelerate                         1.3.0
aiofiles                           22.1.0
aiohappyeyeballs                   2.6.1
aiohttp                            3.11.16
aiosignal                          1.3.2
aiosqlite                          0.21.0
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.4
ale-py                             0.10.1
alembic                            1.15.2
altair                             5.5.0
annotated-types                    0.7.0
annoy                              1.17.3
ansicolors                         1.1.8
antlr4-python3-runtime             4.9.3
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
args    

In [ ]:
print(output.columns.tolist())
